# Vertex AI - Training Job and Serving

This notebook creates a python package for a TensorFlow training project that uses the `<PROJECT_ID>.digits.digits_prepped` BigQuery table. Vertex AI clients are used to setup a Vertex AI custom training job to run the training package.  Then Vertex AI clients are used to upload the model and deploy it to an endpoint for online predictions.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`
- `04 - Vertex AI - Notebook` (helpful to understand)
    - the model created here is stored in a python package in this notebook
    
**Resources**
- Adopted From:
    - https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/custom_job_image_classification_model_for_online_prediction.ipynb
- Using Google Cloud Client Libraries (Python for Vertex AI):
    - https://googleapis.dev/python/aiplatform/latest/index.html

**Overview**

<img src="architectures/statmike-mlops-05b.png">

---
## Setup

Setup the environment:

In [1]:
from google.cloud import aiplatform
from google.cloud import aiplatform_v1beta1
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from datetime import datetime

Define Parameters:

In [2]:
# Locations
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
BUCKET_NAME='gs://{}/digits/model/05_aip_train_job'.format(PROJECT_ID)
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
EXPERIMENT_NAME = '05_AIP_DIGITS_'
JOB_NAME = EXPERIMENT_NAME+TIMESTAMP
MODEL_DIR = '{}/{}'.format(BUCKET_NAME, JOB_NAME)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

# Resources
TRAIN_IMAGE='us-docker.pkg.dev/cloud-aiplatform/training/tf-cpu.2-4:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-3:latest'
TRAIN_COMPUTE='n1-standard-4'
DEPLOY_COMPUTE='n1-standard-4'

# TF Parameters to pass
EPOCHS = 25
BATCH_SIZE = 30

Get Service Account email:

In [3]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials

credentials = GoogleCredentials.get_application_default()

service = discovery.build('iam', 'v1', credentials=credentials)

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/opt/conda/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    "file_cach

In [4]:
request = service.projects().serviceAccounts().list(name="projects/"+PROJECT_ID)
response = request.execute()

INFO:oauth2client.transport:Attempting refresh to obtain initial access_token


In [5]:
SERVICE_ACCOUNT = response['accounts'][0]['email']
SERVICE_ACCOUNT

'691911073727-compute@developer.gserviceaccount.com'

Setup AI Platform Python Clients
- https://googleapis.dev/python/aiplatform/latest/index.html

In [6]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

---
## Get Tensorboard Instance Name
The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

In [10]:
clients['tb'] = aiplatform_v1beta1.TensorboardServiceClient(client_options=client_options)
BASE_RESOURCE_PATH = clients['tb'].common_location_path(PROJECT_ID, REGION)

In [11]:
tensorboard = list(clients['tb'].list_tensorboards(parent=PARENT))[0]
tensorboard

name: "projects/691911073727/locations/us-central1/tensorboards/5984333926728990720"
display_name: "digits_tensorboard"
create_time {
  seconds: 1628770083
  nanos: 850925000
}
update_time {
  seconds: 1628770084
  nanos: 137004000
}
etag: "AMEw9yMQnj1mX94AKtNfec5AchaD831MLqqVgM_39PWz7rSIRnwLTR2ptO98UONQd9Y="
blob_storage_path_prefix: "cloud-ai-platform-f89edcdf-ac24-4545-a8c4-14e268905904"

In [12]:
TENSORBOARD_NAME = tensorboard.name

---
## Training Job

Create a client for the AIP Job Service:

In [13]:
#clients['job'] = aiplatform.gapic.JobServiceClient(client_options=client_options)
clients['job'] = aiplatform_v1beta1.JobServiceClient(client_options=client_options)

Define job parameters to pass to the client:

In [14]:
MACHINE_SPEC = {
    "machine_type": TRAIN_COMPUTE,
    "accelerator_count": 0
}


CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE)
]

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": MACHINE_SPEC,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_cifar.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS
        }
    }
]

JOB_SPEC = {
    "worker_pool_specs": WORKER_POOL_SPEC,
    "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    "tensorboard": TENSORBOARD_NAME,
    "service_account": SERVICE_ACCOUNT
}

CUSTOM_JOB = {
    "display_name": JOB_NAME,
    "job_spec": JOB_SPEC
}

Assemble the Python training package:

In [15]:
!rm -rf custom
!mkdir custom


setup_py = "from setuptools import setup\n\
if __name__ == '__main__':\n\
    setup()"

setup_py = "import setuptools\n\
REQUIRED_PACKAGES = ['tensorflow_io']\n\
setuptools.setup(\n\
    name='trainer',\n\
    version='0.1',\n\
    #install_requires=REQUIRED_PACKAGES,\n\
    packages=setuptools.find_packages(),\n\
    #include_package_data=True,\n\
    description='Digit Training Package')"

!echo "$setup_py" > custom/setup.py

!mkdir custom/trainer
!touch custom/trainer/__init__.py

In [16]:
%%writefile custom/trainer/task.py
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf
from google.cloud import bigquery
import argparse
import os
import sys

parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--model-dir', dest='model_dir', default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--epochs',dest='epochs', default=10, type=int, help='Number of Epochs')
parser.add_argument('--batch_size',dest='batch_size', default=32, type=int, help='Batch Size')
#parser.add_argument('',dest='', default=, type=, help='')
args = parser.parse_args()

# built in parameters for data source:
PROJECT_ID='statmike-mlops'
BQDATASET_ID='digits'
BQTABLE_ID='digits_prepped'

selected_fields = ['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'target']
output_types = ['FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'INT64']

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != 'target':
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

from tensorflow.python.framework import dtypes
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

def transTable(row_dict):
    target=row_dict.pop('target')
    target = tf.one_hot(tf.cast(target,tf.int64),10)
    target = tf.cast(target,tf.float32)
    return(row_dict,target)

client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TRAIN'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
train = table.shuffle(100000).batch(args.batch_size)

client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TEST'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
test = table.batch(args.batch_size)

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()],outputs=tf.keras.layers.Dense(10,activation=tf.nn.softmax)(feature_layer_outputs))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
tf.keras.utils.plot_model(model,show_shapes=True, show_dtype=True)

# setup tensorboard logs
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'], histogram_freq=1) #(log_dir=model_dir+'/logs', histogram_freq=1)

history = model.fit(train,epochs=args.epochs, callbacks=[tensorboard_callback])

model.save(args.model_dir)

Writing custom/trainer/task.py


Store the training package in a Cloud Storage Bucket:

In [17]:
!rm -f custom.tar custom.tar.gz
!tar cvf custom.tar custom
!gzip custom.tar
!gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar.tar.gz

custom/
custom/trainer/
custom/trainer/task.py
custom/trainer/__init__.py
custom/setup.py
Copying file://custom.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.7 KiB/  1.7 KiB]                                                
Operation completed over 1 objects/1.7 KiB.                                      


Submit the training job:

In [18]:
jobID = clients['job'].create_custom_job(parent=PARENT, custom_job=CUSTOM_JOB)

Get information as the job runs:

In [20]:
jobIDresponse = clients['job'].get_custom_job(name=jobID.name)

In [21]:
#response = clients['job'].list_custom_jobs(parent=PARENT)
#response = clients['job'].cancel_custom_job(name=name)

---
## Serving

---
### Upload the Model

Check that model training was successful (and update the path to the model store):

In [22]:
if jobIDresponse.state == aiplatform.gapic.JobState.JOB_STATE_SUCCEEDED:
    MODEL_DIR = MODEL_DIR + "/model"
    
print(MODEL_DIR)

gs://statmike-mlops/digits/model/05_aip_train_job/05_AIP_DIGITS_20210812120620/model


Create a client to the Model Service:

In [23]:
clients['model'] = aiplatform.gapic.ModelServiceClient(client_options=client_options)

Upload the model using the client:

In [24]:
MODEL = {
    "display_name": jobIDresponse.display_name,
    "metadata_schema_uri": "",
    "artifact_uri": MODEL_DIR,
    "container_spec": {
        "image_uri": DEPLOY_IMAGE,
        "command": [],
        "args": [],
        "env": [{"name": "env_name", "value": "env_value"}],
        "ports": [{"container_port": 8080}],
        "predict_route": "",
        "health_route": ""
    }
}

uploaded_model = clients['model'].upload_model(parent=PARENT, model=MODEL)

Review the uploaded models information:

In [25]:
model_info = clients['model'].get_model(name=uploaded_model.result(timeout=180).model)
model_info.name

'projects/691911073727/locations/us-central1/models/464741574828359680'

---
### Endpoint Creation

Create a client to the endpoint service:

In [26]:
clients['endpoint'] = aiplatform.gapic.EndpointServiceClient(client_options=client_options)

Create the endpoint:

In [27]:
ENDPOINT_NAME = 'ENDPOINT_'+JOB_NAME
endpoint = clients['endpoint'].create_endpoint(parent=PARENT, endpoint={"display_name": ENDPOINT_NAME})

In [28]:
endpoint_info = clients['endpoint'].get_endpoint(name=endpoint.result(timeout=180).name)
endpoint_info.name

'projects/691911073727/locations/us-central1/endpoints/2317225153584431104'

---
### Deploy Model to Endpoint

Setup Deployment Parameters:

In [29]:
MACHINE_SPEC = {
    "machine_type": DEPLOY_COMPUTE,
    "accelerator_count": 0,
}
DMODEL = {
        "model": model_info.name,
        "display_name": 'DEPLOYED_'+JOB_NAME,
        "dedicated_resources": {
            "min_replica_count": 1,
            "max_replica_count": 2,
            "machine_spec": MACHINE_SPEC
        }   
}
TRAFFIC = {
    '0' : 100
}

Deploy the Model to the Endpoint:

In [30]:
dmodel = clients['endpoint'].deploy_model(endpoint=endpoint_info.name, deployed_model=DMODEL, traffic_split=TRAFFIC)

In [32]:
dmodel_info = dmodel.result().deployed_model
dmodel_info.id

'2519491312630104064'

In [33]:
clients['endpoint'].get_endpoint(name=endpoint_info.name)

name: "projects/691911073727/locations/us-central1/endpoints/2317225153584431104"
display_name: "ENDPOINT_05_AIP_DIGITS_20210812120620"
deployed_models {
  id: "2519491312630104064"
  model: "projects/691911073727/locations/us-central1/models/464741574828359680"
  display_name: "DEPLOYED_05_AIP_DIGITS_20210812120620"
  create_time {
    seconds: 1628774845
    nanos: 250186000
  }
  dedicated_resources {
    machine_spec {
      machine_type: "n1-standard-4"
    }
    min_replica_count: 1
    max_replica_count: 2
  }
}
traffic_split {
  key: "2519491312630104064"
  value: 100
}
etag: "AMEw9yPm6Q6tMHJJzj70DxAxdhhfckv0YlKwiFOiPubm0w8UgWlur7rK-_hUti-Qm9Mg"
create_time {
  seconds: 1628774806
  nanos: 473858000
}
update_time {
  seconds: 1628775092
  nanos: 785737000
}

---
## Prediction

Create a client to the prediction service:

In [34]:
clients['prediction'] = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

Setup an observation for prediction:

In [35]:
%%bigquery pred
SELECT *
FROM `digits.digits_prepped`
WHERE splits='TEST'

Downloading: 100%|██████████| 388/388 [00:01<00:00, 277.63rows/s]


In [36]:
pred.head(1)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p57,p58,p59,p60,p61,p62,p63,target,target_OE,SPLITS
0,0.0,0.0,0.0,0.0,7.0,16.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,16.0,6.0,0.0,1,Odd,TEST


In [37]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]
#newob

Request prediction from prediction service:

In [38]:
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

response = clients['prediction'].predict(endpoint=endpoint_info.name, instances=[json_format.ParseDict(newob, Value())], parameters=json_format.ParseDict({}, Value()))

In [39]:
response.predictions

[[6.52381758e-08, 0.789791405, 2.92022878e-10, 2.78897587e-05, 2.4280338e-05, 6.61314687e-07, 7.03922165e-09, 1.74794152e-07, 0.000356068485, 0.209799454]]

In [40]:
import numpy as np
np.argmax(response.predictions[0])

1

---
## Remove Resources
see notebook "XX - Cleanup"